In [ ]:
import tensorflow as tf
import os
import numpy as np
import warnings
from tensorflow.keras import models, layers
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_dir = '/content/drive/MyDrive/chestCTscans/Data'
target_var = os.listdir(data_dir)
target_var

In [ ]:
s=256
METRICS = [
      BinaryAccuracy(name='acc'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc')
]

In [ ]:
train_data=ImageDataGenerator(rescale=1./255)
test_data=ImageDataGenerator(rescale=1./255)
valid_data=ImageDataGenerator(rescale=1./255)

train_dir = os.path.join(data_dir ,'train')
train_set = train_data.flow_from_directory(train_dir,target_size=(s,s),batch_size=32,class_mode='categorical')

test_dir=os.path.join(data_dir ,'test')
test_set = test_data.flow_from_directory(test_dir,target_size=(s,s),batch_size=32,class_mode='categorical')

valid_dir=os.path.join(data_dir ,'valid')
valid_set = test_data.flow_from_directory(valid_dir,target_size=(s,s),batch_size=32,class_mode='categorical')

In [ ]:
class_names=list(test_set.class_indices.keys())
class_names

In [ ]:
class_names=list(test_set.class_indices.keys())

plt.figure(figsize=(10, 10))
for _ in range(5):
    img, label = next(test_set)  # Change from test_set.next() to next(test_set)
    plt.subplot(1, 5, _ + 1)
    plt.tight_layout()

# Add code to display or process the images as needed
plt.show()

In [ ]:
cancer_model = models.Sequential()
cancer_model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(s,s,3)))
cancer_model.add(layers.MaxPooling2D((2, 2)))
cancer_model.add(layers.Conv2D(32, (3, 3), activation='relu'))
cancer_model.add(layers.MaxPooling2D((2, 2)))
cancer_model.add(layers.Conv2D(32, (3, 3), activation='relu'))
cancer_model.add(layers.Dropout((0.4)))
cancer_model.add(layers.Flatten())
cancer_model.add(layers.Dense(256, activation='relu', name='Layer_1'))
cancer_model.add(layers.Dropout((0.4)))
cancer_model.add(layers.Dense(128, activation='relu', name='Layer_2'))
cancer_model.add(layers.Dense(4, activation='softmax'))
cancer_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 127, 127, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 32)      18464     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 32)        9248      
                                                                 
 dropout_2 (Dropout)         (None, 60, 60, 32)       

In [ ]:
cancer_model.compile(loss='categorical_crossentropy',
              optimizer="rmsprop",
              metrics=METRICS)
hist = cancer_model.fit(
         train_set,
         validation_data=valid_set,
         batch_size=32,
         epochs=30)

Epoch 1/30
20/20 [==============================] - 157s 8s/step - loss: 2.2468 - acc: 0.7378 - precision: 0.4062 - recall: 0.1060 - auc: 0.5894 - val_loss: 1.2126 - val_acc: 0.7812 - val_precision: 1.0000 - val_recall: 0.1250 - val_auc: 0.6846
Epoch 2/30
20/20 [==============================] - 127s 6s/step - loss: 1.1075 - acc: 0.7830 - precision: 0.6976 - recall: 0.2333 - auc: 0.7750 - val_loss: 1.3553 - val_acc: 0.7326 - val_precision: 0.4419 - val_recall: 0.2639 - val_auc: 0.6801
Epoch 3/30
20/20 [==============================] - 127s 7s/step - loss: 0.8375 - acc: 0.8287 - precision: 0.7506 - recall: 0.4715 - auc: 0.8759 - val_loss: 0.8384 - val_acc: 0.8160 - val_precision: 0.8276 - val_recall: 0.3333 - val_auc: 0.9003
Epoch 4/30
20/20 [==============================] - 126s 6s/step - loss: 0.6090 - acc: 0.8785 - precision: 0.8195 - recall: 0.6591 - auc: 0.9363 - val_loss: 0.9565 - val_acc: 0.7882 - val_precision: 0.5821 - val_recall: 0.5417 - val_auc: 0.8605
Epoch 5/30
20/20 [==